# Calculating the Spin Axis of a Pitch

This tries to follow the paper by Alan Nathan ["Determining the 3D Spin Axis from Statcast Data"](http://baseball.physics.illinois.edu/trackman/SpinAxis.pdf).

Brooks Baseball is the only site I've seen that tabulates spin axis, but they only do it in the "Scatter Charts".

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

### Comparing to Analysis of Andrew Miller 

Trying to compare to the data in the FanGraphs [article about Andrew Miller's slider](https://blogs.fangraphs.com/how-andrew-miller-can-return-to-dominance/) 

In [2]:
data = pd.read_csv("../data/pitch_data_2019.csv")

# andrew miller 
pitcher_id = 453192
data = data[data['pitcher'] == pitcher_id]

# just sliders
data = data[data['pitch_type'] == 'SL']
print(data.shape)

print(data.columns.tolist())

# columns to keep
cols_to_keep = ['game_date', 'release_speed', 'release_spin_rate', 'release_extension',
                'release_pos_x', 'release_pos_z',
                'pfx_x', 'pfx_z', 'plate_x', 'plate_z',
                'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az']

data = data[cols_to_keep]

data.head()

(589, 90)
['index', 'pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_deno

,game_date,release_speed,release_spin_rate,release_extension,release_pos_x,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az
5910,2019-09-28,82.0,2618.0,6.493,2.3130,5.1671,-1.0482,-0.0304,-1.2687,0.9905,-6.0847,-119.3253,-2.8771,-9.0072,21.5664,-32.1420
5911,2019-09-28,81.2,2635.0,6.363,2.4896,5.3631,-1.1783,0.1372,0.2587,2.6697,-2.7488,-118.3768,0.2676,-10.6294,21.3576,-31.1289
5913,2019-09-28,80.8,2609.0,6.397,2.3126,5.4169,-1.2443,-0.5257,-0.0954,2.2549,-2.9967,-117.6492,0.6312,-10.9406,23.2724,-37.3931
5914,2019-09-28,81.0,2520.0,6.643,2.4592,5.2661,-1.1229,-0.4253,0.8071,2.6361,-1.5345,-117.9959,1.5650,-10.3311,21.8613,-36.6784
5916,2019-09-28,80.8,2553.0,6.400,2.2770,5.3245,-1.4039,-0.0410,0.0477,2.3870,-2.2931,-117.6511,0.1906,-12.5927,22.7762,-32.8078


### Comparing to Alan Nathan's Excel Spreadsheet

In [3]:
data.rename(columns={'release_speed': 'v0',
                     'release_pos_x': 'xR',
                     'release_pos_z': 'zR', 
                     }, inplace=True)

data_game_dates = data[['game_date']]

data.head()

,game_date,v0,release_spin_rate,release_extension,xR,zR,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az
5910,2019-09-28,82.0,2618.0,6.493,2.3130,5.1671,-1.0482,-0.0304,-1.2687,0.9905,-6.0847,-119.3253,-2.8771,-9.0072,21.5664,-32.1420
5911,2019-09-28,81.2,2635.0,6.363,2.4896,5.3631,-1.1783,0.1372,0.2587,2.6697,-2.7488,-118.3768,0.2676,-10.6294,21.3576,-31.1289
5913,2019-09-28,80.8,2609.0,6.397,2.3126,5.4169,-1.2443,-0.5257,-0.0954,2.2549,-2.9967,-117.6492,0.6312,-10.9406,23.2724,-37.3931
5914,2019-09-28,81.0,2520.0,6.643,2.4592,5.2661,-1.1229,-0.4253,0.8071,2.6361,-1.5345,-117.9959,1.5650,-10.3311,21.8613,-36.6784
5916,2019-09-28,80.8,2553.0,6.400,2.2770,5.3245,-1.4039,-0.0410,0.0477,2.3870,-2.2931,-117.6511,0.1906,-12.5927,22.7762,-32.8078


In [12]:
def compute_spin_angle_phi(data):
    
# the release point y coordinate
data['yR'] = 60.5 - data['release_extension']

# find speed at release (initial velocity is given at 50 ft)
data['tR'] = (-data['vy0'] - np.sqrt(data['vy0']*data['vy0'] - 2*data['ay']*(50 - data['yR']))) / data['ay']
data['vxR'] = data['vx0'] + data['ax']*data['tR']
data['vyR'] = data['vy0'] + data['ay']*data['tR']
data['vzR'] = data['vz0'] + data['az']*data['tR']
data['dv0'] = data['v0'] - np.sqrt(data['vxR']**2 + data['vyR']**2 + data['vzR']**2)/1.467

# calculate x movement
data['tf'] = (-data['vyR'] - np.sqrt(data['vyR']**2 - 2*data['ay']*(data['yR'] - 17/12)))/data['ay']
data['calculate_x_mvt'] = data['plate_x'] - data['xR'] - (data['vxR']/data['vyR'])*(17/12 - data['yR'])
data['calculate_z_mvt'] = data['plate_z'] - data['zR'] - (data['vzR']/data['vyR'])*(17/12 - data['yR']) + 0.5*(32.174)*data['tf']**2

# average velocities
data['vxbar'] = (2*data['vxR'] + data['ax']*data['tf'])/2
data['vybar'] = (2*data['vyR'] + data['ay']*data['tf'])/2
data['vzbar'] = (2*data['vzR'] + data['az']*data['tf'])/2
data['vbar'] = np.sqrt(data['vxbar']**2 + data['vybar']**2 + data['vzbar']**2)

# drag acceleration
data['adrag'] = -(data['ax']*data['vxbar'] + data['ay']*data['vybar'] + (data['az'] + 32.174)*data['vzbar'])/data['vbar']
data['Cd'] = data['adrag']/8.598E-02/data['vbar']**2

# Magnus acceleration
data['amagx'] = data['ax'] + data['adrag']*data['vxbar']/data['vbar']
data['amagy'] = data['ay'] + data['adrag']*data['vybar']/data['vbar']
data['amagz'] = data['az'] + data['adrag']*data['vzbar']/data['vbar'] + 32.174
data['amag'] = np.sqrt(data['amagx']**2 + data['amagy']**2 + data['amagz']**2)

data['Mx'] = 0.5 * data['amagx'] * data['tf']**2 * 12
data['Mz'] = 0.5 * data['amagz'] * data['tf']**2 * 12

data['Cl'] = data['amag']/8.598E-02/data['vbar']**2

data['S'] = 0.4 * data['Cl'] / (1 - 2.32 * data['Cl']) 

data['spinT'] = 78.92 * data['S'] * data['vbar']
data['spinTx'] = data['spinT'] * (data['vybar']*data['amagz'] - data['vzbar']*data['amagy'])/(data['amag']*data['vbar'])
data['spinTy'] = data['spinT'] * (data['vzbar']*data['amagx'] - data['vxbar']*data['amagz'])/(data['amag']*data['vbar'])
data['spinTz'] = data['spinT'] * (data['vxbar']*data['amagy'] - data['vybar']*data['amagx'])/(data['amag']*data['vbar'])
data['spin_check'] = np.sqrt(data['spinTx']**2 + data['spinTy']**2 + data['spinTz']**2) - data['spinT']

def compute_phi(row):
    if row['amagz'] > 0:
        return -np.arctan(row['amagx']/row['amagz']) * 180 / 3.14 + 180
    else:
        return 180 - np.arctan(row['amagx']/row['amagz']) * 180 / 3.14 + 180
data['phi'] = data.apply(compute_phi, axis=1)

data = data[['v0', 'tR', 'vxR', 'vyR', 'vzR', 'dv0', 'xR', 'yR', 'zR', 
             'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'calculate_x_mvt', 'calculate_z_mvt',
             'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'release_spin_rate', 'release_extension', 
             'tf', 'vxbar', 'vybar', 'vzbar', 'vbar', 'adrag', 'Cd', 'amagx', 'amagy', 'amagz', 'amag',
             'Mx', 'Mz', 'Cl', 'S', 'spinT', 'spinTx', 'spinTy', 'spinTz', 'spin_check', 'phi']]

data_spin_axis = data[['phi', 'v0', 'release_spin_rate']]

data

,v0,tR,vxR,vyR,vzR,dv0,xR,yR,zR,pfx_x,pfx_z,plate_x,plate_z,calculate_x_mvt,calculate_z_mvt,vx0,vy0,vz0,ax,ay,az,release_spin_rate,release_extension,tf,vxbar,vybar,vzbar,vbar,adrag,Cd,amagx,amagy,amagz,amag,Mx,Mz,Cl,S,spinT,spinTx,spinTy,spinTz,spin_check,phi
5910,82.0,-0.033479,-5.783146,-120.047325,-1.801012,0.064051,2.3130,54.007,5.1671,-1.0482,-0.0304,-1.2687,0.9905,-1.048219,-0.030425,-6.0847,-119.3253,-2.8771,-9.0072,21.5664,-32.1420,2618.0,6.493,0.456825,-7.840505,-115.121285,-9.142654,115.749609,20.841740,0.018092,-10.418952,0.837795,-1.614216,10.576491,-13.045955,-2.021219,0.009181,0.003752,34.278547,5.417767,2.312834,-33.768585,0.000000e+00,278.765674
5911,81.2,-0.034838,-2.378490,-119.120861,1.352076,-0.021729,2.4896,54.137,5.3631,-1.1783,0.1372,0.2587,2.6697,-1.178231,0.137225,-2.7488,-118.3768,0.2676,-10.6294,21.3576,-31.1289,2635.0,6.363,0.461687,-4.832219,-114.190596,-5.833831,114.441584,20.915216,0.018574,-11.512531,0.488255,-0.021084,11.522899,-14.723724,-0.026965,0.010233,0.004193,37.867107,0.150930,1.925670,-37.817811,1.421085e-14,270.059337
5913,80.8,-0.034755,-2.616455,-118.458041,1.930812,0.021088,2.3126,54.103,5.4169,-1.2443,-0.5257,-0.0954,2.2549,-1.244285,-0.525736,-2.9967,-117.6492,0.6312,-10.9406,23.2724,-37.3931,2609.0,6.397,0.466109,-5.166213,-113.034301,-6.783823,113.355473,22.395501,0.020271,-11.961282,0.940353,-6.559372,13.674132,-15.592094,-8.550449,0.012377,0.005097,45.599505,21.999406,1.390195,-39.917519,0.000000e+00,298.708571
5914,81.0,-0.032589,-1.197818,-118.708342,2.760319,0.054893,2.4592,53.857,5.2661,-1.1229,-0.4253,0.8071,2.6361,-1.122955,-0.425270,-1.5345,-117.9959,1.5650,-10.3311,21.8613,-36.6784,2520.0,6.643,0.461357,-3.580980,-113.665410,-5.700599,113.864594,21.272639,0.019083,-11.000113,0.625873,-5.569409,12.345547,-14.048263,-7.112701,0.011075,0.004547,40.857863,18.503541,1.242933,-36.406580,7.105427e-15,296.821278
5916,80.8,-0.034732,-1.855730,-118.442164,1.330082,0.047329,2.2770,54.100,5.3245,-1.4039,-0.0410,0.0477,2.3870,-1.403867,-0.040977,-2.2931,-117.6511,0.1906,-12.5927,22.7762,-32.8078,2553.0,6.400,0.465650,-4.787626,-113.139293,-6.308397,113.416122,22.153779,0.020031,-13.527876,0.676494,-1.866031,13.672715,-17.599504,-2.427670,0.012363,0.005091,45.568817,6.329388,2.245236,-45.071220,-7.105427e-15,277.812134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725279,83.1,-0.034445,-5.857446,-121.802698,1.264519,-0.028844,2.1064,54.183,5.4242,-0.9022,-0.2689,-1.3333,2.4354,-0.902187,-0.268963,-6.1287,-121.0806,0.0576,-7.8751,20.9641,-35.0395,2539.0,6.317,0.450692,-7.632067,-117.078524,-6.631489,117.514280,20.213204,0.017024,-9.187864,0.825849,-4.006158,10.057245,-11.197606,-4.882460,0.008470,0.003456,32.052166,12.868699,0.823195,-29.343828,1.421085e-14,293.524809
725280,84.4,-0.035214,-4.659863,-123.732762,-0.882258,-0.006013,2.2060,54.343,5.3707,-0.5673,0.0563,-0.3545,1.8495,-0.567257,0.056193,-4.8306,-122.9330,-1.9896,-4.8486,22.7117,-31.4464,2518.0,6.157,0.446003,-5.741109,-118.668014,-7.894858,119.068832,22.449706,0.018417,-5.931051,0.337565,-0.760928,5.989184,-7.078793,-0.908178,0.004913,0.001988,18.680914,2.435239,1.112177,-18.488083,0.000000e+00,277.268916
725282,83.9,-0.036263,-4.685708,-123.000215,-0.982215,-0.008216,2.1775,54.445,5.4870,-0.9127,-0.4607,-0.7552,1.3398,-0.912579,-0.460744,-4.9798,-122.1537,-2.3069,-8.1100,23.3438,-36.5300,2638.0,6.055,0.450372,-6.511964,-117.743523,-9.208251,118.282436,22.451838,0.018664,-9.346072,0.994256,-6.103869,11.206909,-11.374237,-7.428452,0.009316,0.003809,35.555354,19.522643,1.242224,-29.690175,-7.105427e-15,303.119499
725285,83.4,-0.034642,-3.522100,-122.276938,-0.453165,0.013161,2.3309,54.223,5.3941,-0.8666,-0.0801,-0.0568,1.8647,-0.866651,-0.080042,-3.7974,-121.5285,-1.5924,-7.9469,21.6047,-32.8856,2718.0,6.277,0.449726,-5.309065,-117.418838,-7.847923,117.800507,21.129141,0.017709,-8.899154,0.

In [13]:
data_spin_dates = pd.merge(data_game_dates, data_spin_axis, left_index=True, right_index=True)

data_spin_dates

,game_date,phi,v0,release_spin_rate
5910,2019-09-28,278.765674,82.0,2618.0
5911,2019-09-28,270.059337,81.2,2635.0
5913,2019-09-28,298.708571,80.8,2609.0
5914,2019-09-28,296.821278,81.0,2520.0
5916,2019-09-28,277.812134,80.8,2553.0
...,...,...,...,...
725279,2019-03-29,293.524809,83.1,2539.0
725280,2019-03-29,277.268916,84.4,2518.0
725282,2019-03-29,303.119499,83.9,2638.0
725285,2019-03-29,283.356000,83.4,2718.0


In [14]:
data_spin_dates['game_date'] = pd.to_datetime(data_spin_dates['game_date'])

data_spin_dates['month'] = data_spin_dates['game_date'].dt.month

data_spin_dates['month'] = data_spin_dates['month'].apply(lambda x: 4 if x == 3 else x)

data_spin_dates

,game_date,phi,v0,release_spin_rate,month
5910,2019-09-28,278.765674,82.0,2618.0,9
5911,2019-09-28,270.059337,81.2,2635.0,9
5913,2019-09-28,298.708571,80.8,2609.0,9
5914,2019-09-28,296.821278,81.0,2520.0,9
5916,2019-09-28,277.812134,80.8,2553.0,9
...,...,...,...,...,...
725279,2019-03-29,293.524809,83.1,2539.0,4
725280,2019-03-29,277.268916,84.4,2518.0,4
725282,2019-03-29,303.119499,83.9,2638.0,4
725285,2019-03-29,283.356000,83.4,2718.0,4


In [16]:
data_spin_dates = data_spin_dates[['month', 'phi', 'v0', 'release_spin_rate']]

data_spin_avg = pd.DataFrame(data_spin_dates.groupby('month')[['phi', 'v0', 'release_spin_rate']].mean())

round(data_spin_avg,1)

,phi,v0,release_spin_rate
month,,,
4,281.0,82.9,2553.9
5,282.7,82.0,2555.3
6,278.5,82.1,2562.2
7,276.8,82.7,2638.6
8,279.4,81.7,2615.5
9,280.6,82.4,2648.3


In [39]:
def compute_spin_angle_phi(row):

    # find speed at release (initial velocity is given at 50 ft)
    tR = (-row['vy0'] - np.sqrt(row['vy0']**2 - 2*row['ay']*(50 - row['yR']))) / row['ay']
    vxR = row['vx0'] + row['ax']*tR
    vyR = row['vy0'] + row['ay']*tR
    vzR = row['vz0'] + row['az']*tR
    dv0 = row['v0'] - np.sqrt(vxR**2 + vyR**2 + vzR**2)/1.467

    # calculate x movement
    tf = (-vyR - np.sqrt(vyR**2 - 2*row['ay']*(row['yR'] - 17/12)))/row['ay']
    calculate_x_mvt = row['plate_x'] - row['xR'] - (vxR/vyR)*(17/12 - row['yR'])
    calculate_z_mvt = row['plate_z'] - row['zR'] - (vzR/vyR)*(17/12 - row['yR']) + 0.5*(32.174)*tf**2

    # average velocities
    vxbar = (2*vxR + row['ax']*tf)/2
    vybar = (2*vyR + row['ay']*tf)/2
    vzbar = (2*vzR + row['az']*tf)/2
    vbar = np.sqrt(vxbar**2 + vybar**2 + vzbar**2)

    # drag acceleration
    adrag = -(row['ax']*vxbar + row['ay']*vybar + (row['az'] + 32.174)*vzbar) / vbar
    Cd = adrag/8.598E-02/vbar**2

    # Magnus acceleration
    amagx = row['ax'] + adrag*vxbar / vbar
    amagy = row['ay'] + adrag*vybar / vbar
    amagz = row['az'] + adrag*vzbar / vbar + 32.174
    amag = np.sqrt(amagx**2 + amagy**2 + amagz**2)

    def compute_phi(amagx, amagz):
        if amagz > 0:
            return -np.arctan(amagx / amagz) * 180 / 3.14 + 180
        else:
            return 180 - np.arctan(amagx / amagz) * 180 / 3.14 + 180
    phi = compute_phi(amagx, amagz)

    return round(phi, 0)

In [40]:
test_data = pd.read_csv("/home/chris/Downloads/savant_data (1).csv")

print(test_data.columns.tolist())

test_data.head()

['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,SI,2018-06-30,86.3,-3.0782,4.9832,Sergio Romo,503556,489265,field_out,hit_into_play,NaN,NaN,NaN,NaN,7,"Marwin Gonzalez grounds out, second baseman Da...",R,L,R,TB,HOU,X,4.0,ground_ball,2,1,2018,-1.5977,0.0639,-0.7199,2.1652,NaN,NaN,NaN,2,9,Top,144.68,151.06,NaN,NaN,467092,NaN,180630_224950,8.8771,-125.4281,-0.5676,-18.7128,26.4515,-31.6015,3.5348,1.5873,25.0,86.7,-2.8,86.236,2281.0,6.410,530644,489265,467092,543068,621002,622110,588751,621563,595281,460576,54.0880,0.232,0.198,0.0,1.0,0.0,0.0,2.0,70,4,Sinker,5,2,2,5,2,5,2,5,Infield shift,Strategic
1,SL,2018-06-30,77.7,-2.9818,4.8713,Sergio Romo,503556,489265,NaN,ball,NaN,NaN,NaN,NaN,14,NaN,R,L,R,TB,HOU,B,NaN,NaN,1,1,2018,1.5983,1.0293,1.1738,2.2696,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,467092,NaN,180630_224927,5.9680,-112.9579,-0.4161,12.1572,23.5430,-23.7193,3.6172,1.7998,NaN,NaN,NaN,76.863,2914.0,6.069,530644,489265,467092,543068,621002,622110,588751,621563,595281,460576,54.4292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70,3,Slider,5,2,2,5,2,5,2,5,Infield shift,Strategic
2,CH,2018-06-30,81.3,-2.7906,5.0780,Sergio Romo,503556,489265,NaN,ball,NaN,NaN,NaN,NaN,11,NaN,R,L,R,TB,HOU,B,NaN,NaN,0,1,2018,-1.5459,0.7456,-1.3176,3.4476,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,467092,NaN,180630_224909,6.1690,-118.1219,1.5629,-15.4466,22.9279,-25.8441,3.7449,1.8415,NaN,NaN,NaN,80.618,2042.0,5.949,530644,489265,467092,543068,621002,622110,588751,621563,595281,460576,54.5486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70,2,Changeup,5,2,2,5,2,5,2,5,Infield shift,Strategic
3,SI,2018-06-30,87.3,-3.1056,4.9332,Sergio Romo,503556,489265,NaN,foul,NaN,NaN,NaN,NaN,1,NaN,R,L,R,TB,HOU,S,NaN,NaN,0,0,2018,-1.4570,0.8234,-0.5870,3.0868,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,467092,NaN,180630_224847,8.9821,-126.7092,0.0950,-17.4420,27.4220,-23.6762,3.5327,1.5834,136.0,68.6,21.7,86.603,2274.0,6.120,530644,489265,467092,543068,621002,622110,588751,621563,595281,460576,54.3777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70,1,Sinker,5,2,2,5,2,5,2,5,Infield shift,Strategic
4,FF,2018-06-30,87.3,-2.7886,5.0168,Sergio Romo,643603,489265,field_out,hit_into_play,NaN,NaN,NaN,NaN,6,"Tyler White grounds out, shortstop Adeiny Hech...",R,R,R,TB,HOU,X,6.0,ground_ball,0,0,2018,-1.1788,1.0520,0.2941,2.3622,NaN,NaN,NaN,1,9,Top,105.31,157.12,NaN,NaN,467092,NaN,180630_224808,9.7797,-126.6707,-2.3184,-14.6474,26.7831,-20.6888,2.9392,1.2514,68.0,101.3,2.3,86.754,2180.0,6.179,530644,489265,467092,543068,621002,622110,588751,621563,595281,460576,54.3188,0.534,0.492,0.0,1.0,0.0,0.0,4.0,69,1,4-Seam Fastball,5,2,2,5,2,5,2,5,Infield shift,Standard


In [41]:
cols_to_keep = ['pitch_type', c]

test_data = test_data[cols_to_keep]

test_data.rename(columns={'release_speed': 'v0', 
                  'release_pos_x': 'xR',
                  'release_pos_y': 'yR',
                  'release_pos_z': 'zR'}, inplace=True)

test_data = test_data[(test_data['v0'] == 85.1) & (test_data['xR'] == -2.7068)]

test_data.head()

,pitch_type,v0,xR,zR,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,release_spin_rate,release_extension,yR
3893,CH,85.1,-2.7068,5.5999,-0.0736,0.1638,0.7974,1.7618,8.1994,-123.4441,-2.884,-2.4569,26.067,-30.2297,2235.0,5.41,55.088


In [42]:
test_data['phi'] = test_data.apply(compute_spin_angle_phi, axis=1)

test_data.head()

,pitch_type,v0,xR,zR,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,release_spin_rate,release_extension,yR,phi
3893,CH,85.1,-2.7068,5.5999,-0.0736,0.1638,0.7974,1.7618,8.1994,-123.4441,-2.884,-2.4569,26.067,-30.2297,2235.0,5.41,55.088,264.0
